<a href="https://colab.research.google.com/github/isegura/BasicNLP/blob/master/NER_by_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning based approach for NER (using Conditional Random Field (CRF)).


https://www.aitimejournal.com/@akshay.chavan/complete-tutorial-on-named-entity-recognition-ner-using-python-and-keras


In NLP, NER is a method of extracting the relevant information from a large corpus and classifying those entities into predefined categories such as location, organization, name and so on. This is a simple example and one can come up with complex entity recognition related to domain-specific with the problem at hand.

<img src='https://d2ueix13hy5h3i.cloudfront.net/wp-content/uploads/2019/06/3.png' width=800>

In this tutorial, we will use the following dataset

https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus#ner_dataset.csv

This dataset is extracted from GMB(Groningen Meaning Bank) corpus which is tagged, annotated and built specifically to train the classifier to predict named entities such as name, location, etc.
All the entities are labeled using the BIO scheme, where each entity label is prefixed with either B or I letter. B- denotes the beginning and I- inside of an entity. The words which are not of interest are labeled with 0 – tag.

<img src='https://d2ueix13hy5h3i.cloudfront.net/wp-content/uploads/2019/06/Capture1.png' width=350>


Before to load the dataset, we need to mount our folder in google drive:

In [6]:
#from google.colab import drive
#drive.mount("/content/drive/")

path='drive/My Drive/Colab Notebooks/TESI/4-NER/'
!ls 'drive/My Drive/Colab Notebooks/TESI/4-NER/'

 chemdner_corpus		       IntroNER-spacy.ipynb   Untitled0.ipynb
 data				      'NER by CRF.ipynb'
'Dictionary based NER (spacy).ipynb'   resources


First, we must load the libraries that we will use in this notebook:

In [8]:
!pip install sklearn-crfsuite
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

     |████████████████████████████████| 757kB 4.1MB/s 


Now, we can load the dataset. Sentence # indicates the sentence number and each sentence comprises of words that are labeled using the BIO scheme in the tag column.

This particular dataset has 47959 sentences and 35178 unique words. 

Observations :
- There are total 47959 sentences in the dataset.
- Number unique words in the dataset are 35178.
- Total 17 lables (Tags).


In [18]:
import pandas as pd
path_data=path+'data/ner_dataset.csv'

#Reading the csv file
df = pd.read_csv(path_data, encoding = "ISO-8859-1")
print(df.head(15))
print()
print('some statistics about the dataset:')
df.describe()


     Sentence #           Word  POS    Tag
0   Sentence: 1      Thousands  NNS      O
1           NaN             of   IN      O
2           NaN  demonstrators  NNS      O
3           NaN           have  VBP      O
4           NaN        marched  VBN      O
5           NaN        through   IN      O
6           NaN         London  NNP  B-geo
7           NaN             to   TO      O
8           NaN        protest   VB      O
9           NaN            the   DT      O
10          NaN            war   NN      O
11          NaN             in   IN      O
12          NaN           Iraq  NNP  B-geo
13          NaN            and   CC      O
14          NaN         demand   VB      O

some statistics about the dataset:


,Sentence #,Word,POS,Tag
count,47959,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 534,the,NN,O
freq,1,52573,145807,887908


Let us to show the set of tags, which are the classes to classify the tokens based on the IOB schema:


In [21]:
##Displaying the unique Tags
df['Tag'].unique()


array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [22]:

#Checking null values, if any.
df.isnull().sum()

Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64

There are lots of missing values in 'Sentence #' attribute. So we will use pandas fillna technique and use 'ffill' method which propagates last valid observation forward to next.



In [27]:
df = df.fillna(method = 'ffill')
df.head(100)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
95,Sentence: 5,'s,POS,O
96,Sentence: 5,ruling,VBG,O
97,Sentence: 5,Labor,NNP,B-org
98,Sentence: 5,Party,NNP,I-org


In [0]:
# This is a class to get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [29]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [30]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)
#getting all sentences
sentences = getter.sentences


[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


## Feature Preparation
These are the default features used by the NER in nltk. We can also modify it for our customization.



In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]


We must split the dataset to obtain training-test dataset:

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


## CRF

CRFs are used for predicting the sequences that use the contextual information to add information which will be used by the model to make a correct prediction.

Below is the formula for CRF where y is the output variable and X is input sequence.



<img src='https://d2ueix13hy5h3i.cloudfront.net/wp-content/uploads/2019/06/CodeCogsEqn1-6.png' width=500>

https://www.aitimejournal.com/@akshay.chavan/introduction-to-conditional-random-fields-crfs

The output sequence is modeled as the normalized product of the feature function.


This process can take several minutes....


In [34]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [35]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

report = flat_classification_report(y_test, y_pred)
print(report)


0.971036183836736
              precision    recall  f1-score   support

       B-art       0.56      0.14      0.23       104
       B-eve       0.63      0.47      0.54        68
       B-geo       0.85      0.91      0.88      7448
       B-gpe       0.97      0.94      0.96      3179
       B-nat       0.85      0.40      0.55        57
       B-org       0.80      0.73      0.77      4020
       B-per       0.85      0.83      0.84      3328
       B-tim       0.92      0.88      0.90      4139
       I-art       0.10      0.03      0.05        64
       I-eve       0.43      0.38      0.41        52
       I-geo       0.81      0.79      0.80      1487
       I-gpe       0.97      0.62      0.76        50
       I-nat       0.71      0.45      0.56        11
       I-org       0.82      0.79      0.80      3331
       I-per       0.86      0.89      0.87      3455
       I-tim       0.85      0.75      0.79      1262
           O       0.99      0.99      0.99    177356

    accu